In [2]:
#3 pointer is worth 3, 
#2 point is worth 2, 
#FT is worth 1, 
#Rebounds are worth 1.2, 
#Assists are worth 1.5, 
#Blocks are worth 2, 
#Steals are worth 2, 
#Turnovers are worth -1

import pandas as pd 

#taking in the pic16b_project.csv file and giving back a new csv that has the modified fantasy stats conversion
def fantasy_stats(nba_df):
    columns = ['Year', 'PLAYER', 'GP', 'FGM', 'FG3M','FTM', 'REB', 'AST', 'STL', 'BLK', 'TOV']
    
    
    nba_df['FTM'] = nba_df['FTM'] * 1
    nba_df['FGM'] = nba_df['FGM'] * 2
    nba_df['FG3M'] = nba_df['FG3M'] * 3
    nba_df['REB'] = nba_df['REB'] * 1.2 
    nba_df['AST'] = nba_df['AST'] * 1.5  
    nba_df['STL'] = nba_df['STL'] * 2  
    nba_df['BLK'] = nba_df['BLK'] * 2  
    nba_df['TOV'] = nba_df['TOV'] * -1  
    
    # Calculate total fantasy points
    nba_df['Fantasy Points'] = nba_df['GP'] * (
        nba_df['FTM'] +
        nba_df['FG3M'] +
        nba_df['FGM'] +
        nba_df['REB'] + 
        nba_df['AST'] + 
        nba_df['STL'] + 
        nba_df['BLK'] + 
        nba_df['TOV'] 
    )
    
    # Sort the DataFrame by Fantasy Points in descending order
    nba_df = nba_df.sort_values(by='Fantasy Points', ascending=False)
    
    return nba_df[['Year', 'PLAYER', 'FGM','FG3M','FTM', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'Fantasy Points']]


In [3]:
nba_api_df = pd.read_csv('pic16b_project.csv')
sorted_fantasy_df = fantasy_stats(nba_api_df)

In [49]:
print(sorted_fantasy_df)

         Year                  PLAYER   FGM  FG3M   FTM    REB    AST  STL  \
0     2018-19            James Harden  21.6  14.4   9.7   7.92  11.25  4.0   
1229  2023-24             Luka Dončić  23.0  12.3   6.8  11.04  14.70  2.8   
1238  2023-24            Nikola Jokić  20.8   3.3   4.5  14.88  13.50  2.8   
761   2021-22            Nikola Jokić  20.6   3.9   5.1  16.56  11.85  3.0   
258   2019-20            James Harden  19.8  13.2  10.2   7.92  11.25  3.6   
...       ...                     ...   ...   ...   ...    ...    ...  ...   
754   2020-21  Thanasis Antetokounmpo   2.4   0.3   0.4   2.64   1.20  0.8   
517   2019-20            Goga Bitadze   2.6   0.3   0.4   2.40   0.60  0.4   
1226  2022-23            Anthony Gill   2.4   0.3   0.8   2.04   0.90  0.2   
755   2020-21                Miye Oni   1.2   1.8   0.1   1.92   0.75  0.4   
257   2018-19     Sindarius Thornwell   0.6   0.0   0.4   0.84   0.45  0.4   

      BLK  TOV  Fantasy Points  
0     1.4 -5.0         5091.06

In [65]:
from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

#setting up our data 
df = sorted_fantasy_df

#initialize the website
app = Dash()

#create the layout
app.layout = html.Div([
    html.H1("Basketball Fantasy Stats", style={'text-align': 'center'}),
    html.Hr(),
    dcc.Input(
        id='player-name-input',
        type='text',
        value='James Harden',
        placeholder='Enter Player Name',
        style={'margin-bottom': '20px'}
    ),
    dcc.Graph(figure={}, id='player-fantasy-graph')
    ])
        #this should be turned into a function so that it doesn't have to be repeated multiple times 
        
# Callback to update the graph
@app.callback(
    Output(component_id='player-fantasy-graph', component_property='figure'),
    Input(component_id='player-name-input', component_property='value')
)
#Making the graph
def update_graph(player_name):
        # Filter the DataFrame for the selected player
        filtered_df = df[df['PLAYER'].str.contains(player_name, case=False, na=False)]
        # Create a bar chart for the player's fantasy stats over the years
        filtered_df = filtered_df.sort_values(by='Year')
        fig = px.bar(
            filtered_df,
            x='Year',
            y='Fantasy Points',
            title=f"{player_name}'s Fantasy Points Over the Years"
        )

        return fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(mode = 'inline')

In [4]:
from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

#setting up our data 
df = sorted_fantasy_df

#initialize the website
app = Dash()

#create the layout
app.layout = html.Div([
    html.H1("Basketball Fantasy Stats", style={'text-align': 'center'}),
    html.Hr(),
    dcc.RadioItems(
#         id='mode-selector',
#         options=[
#             {'label': 'Single Player', 'value': 'single'},
#             {'label': 'Compare Two Players', 'value': 'compare'}
#         ],
#         value='single',
#         style={'margin-bottom': '20px'}
#     ),
        options=['Single Player', 'Compare Players'],
        value='Compare Players',
        id='mode-selector'
    ),
#     dcc.Input(
#         id='player-name-input',
#         type='text',
#         value='James Harden',
#         placeholder='Enter Player Name',
#         style={'margin-bottom': '10px'}
#     ),
#     dcc.Input(
#         id='second-player-name-input',
#         type='text',
#         value='',
#         placeholder='Enter Second Player Name for Comparison',
#         style={'margin-bottom': '20px'}
#     ), 
    html.Div([
        html.Label("Player Name:"),
        dcc.Input(id='player-name-input', value='DeMar DeRozan', type='text'),
        html.Label("Second Player Name:"),
        dcc.Input(id='second-player-name-input', value = 'James Harden', type='text')
    ]),
    
    dcc.Graph(figure={}, id='player-fantasy-graph')
])


@app.callback(
    [Output('player-fantasy-graph', 'figure')],
    [Input('mode-selector', 'value'),
     Input('player-name-input', 'value'),
     Input('second-player-name-input', 'value')]
)

  
def update_graph(mode, player_name, second_player_name):
    filtered_df = df.sort_values(by='Year')
    fig = px.bar(title="Enter player names to see their fantasy stats")
    if mode == 'Single Player' and player_name:
        # Filter for the single player
        filtered_df = df[df['PLAYER'].str.contains(player_name, case=False, na=False)]
        fig = px.bar(
            filtered_df,
            x='Year',
            y='Fantasy Points',
            title=f"{player_name}'s Fantasy Points Over the Years"
        )
    elif mode == 'Compare Players' and player_name and second_player_name:
        # Filter for both players
        filtered_df = df[df['PLAYER'].str.contains(player_name + '|' + second_player_name, case=False, na=False)]
        fig = px.bar(
            filtered_df,
            x='Year',
            y='Fantasy Points',
            color='PLAYER',
            barmode='group',
            title=f"Comparison of {player_name} and {second_player_name} Fantasy Points"
        )

            
    return [fig] 

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas as pd

def predict_next_fantasy_point(df):
    predictions = []
    
    # Ensure the input data is sorted by 'Year' for proper chronological order
    df = df.sort_values(by=['PLAYER', 'Year'])
    
    for player_name in df['PLAYER'].unique():
        # Filter data for the specific player
        player_data = df[df['PLAYER'] == player_name].copy()
        
        # Ensure there's enough data for training
        if len(player_data) < 2:
            continue  # Skip this player if not enough data
        
        # Create the 'NextYearPoints' column (target)
        player_data['NextYearPoints'] = player_data['Fantasy Points'].shift(-1)
        
        # Drop the last row (no target value available for it)
        player_data = player_data.dropna()
        
        # Ensure there's enough data left after dropping NaNs
        if len(player_data) < 2:
            continue
        
        # Define X (features) and y (target)
        X = player_data['Fantasy Points'].values.reshape(-1, 1)
        y = player_data['NextYearPoints'].values
        
        # Split the data into training and testing sets (80/20 split)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Initialize and train the model (simple linear regression)
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predict the next year's fantasy points based on the most recent data point
        last_year_points = player_data['Fantasy Points'].iloc[-1]
        next_year_prediction = model.predict([[last_year_points]])[0]
        
        # Append the prediction to the list
        predictions.append({'PLAYER': player_name, 'Predicted Fantasy Points': next_year_prediction})
    
    # Create a DataFrame from the predictions and sort by predicted points in descending order
    predictions_df = pd.DataFrame(predictions)
    draft_order_df = predictions_df.sort_values(by='Predicted Fantasy Points', ascending=False).reset_index(drop=True)
    
    return draft_order_df


In [20]:
draft_order_df = predict_next_fantasy_point(sorted_fantasy_df)
print(draft_order_df)

                      PLAYER  Predicted Fantasy Points
0                Luka Dončić               4487.650928
1               Nikola Jokić               4024.659698
2               Jayson Tatum               4018.447294
3    Shai Gilgeous-Alexander               3915.000000
4      Giannis Antetokounmpo               3850.151138
..                       ...                       ...
248      Juancho Hernangomez                692.160000
249             Jaxson Hayes                689.500000
250           Sterling Brown                670.800000
251           Troy Brown Jr.                624.220000
252              Cody Martin                617.240000

[253 rows x 2 columns]


In [21]:
# Assuming df_fantasy contains historical data with columns: 'PLAYER', 'Year', 'Fantasy Points'
# And df_predictions contains the predicted stats with columns: 'PLAYER', 'Year', 'Predicted Fantasy Points'

df = sorted_fantasy_df

# Pivot the historical data so each player is one row with 'Year' columns
pivoted_df = df.pivot(index='PLAYER', columns='Year', values='Fantasy Points')

# Optionally, rename the columns to indicate which are historical points
pivoted_df.columns = [f'Fantasy Points {year}' for year in pivoted_df.columns]

# Reset index to make 'PLAYER' a column again
pivoted_df.reset_index(inplace=True)

# Merge the pivoted historical data with the predicted stats
merged_df = pivoted_df.merge(draft_order_df, on='PLAYER', how='left')

# Rename the predicted column for clarity if needed
merged_df.rename(columns={'Predicted Fantasy Points': 'Fantasy Points (Predicted)'}, inplace=True)

# Display the final DataFrame
print(merged_df)


              PLAYER  Fantasy Points 2018-19  Fantasy Points 2019-20  \
0         AJ Griffin                     NaN                     NaN   
1       Aaron Gordon                 2677.74                 2032.98   
2      Aaron Holiday                     NaN                 1364.88   
3      Aaron Nesmith                     NaN                     NaN   
4      Aaron Wiggins                     NaN                     NaN   
..               ...                     ...                     ...   
534      Zach LaVine                 2475.27                 2667.60   
535    Zaza Pachulia                  777.24                     NaN   
536       Zeke Nnaji                     NaN                     NaN   
537  Ziaire Williams                     NaN                     NaN   
538  Zion Williamson                     NaN                     NaN   

     Fantasy Points 2020-21  Fantasy Points 2021-22  Fantasy Points 2022-23  \
0                       NaN                     NaN     

The code below needs to be edited for our functionality, names, etc. If this works it should be input into the dash app 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load player performance data
data = merged_df

# Create features for trade prediction
data['FantasyPoints'] = data['FantasyPointsPerGame'] #Need to maybe divide fantasy points by number of games played
data['LastSeasonTotalPoints'] = data['TotalPoints'] 
data['Consistency'] = data['FantasyPointsStdDev'] #Need to add Standard Dev to the data frame

# Prepare the feature matrix and target variable
X = data[['FantasyPoints', 'LastSeasonTotalPoints', 'Consistency']]
y = data['TradeAccepted']  # 1 for accepted trades, 0 for rejected

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate model performance
y_pred = model.predict(X_test)
print("Model accuracy:", accuracy_score(y_test, y_pred))

# Example of predicting for a new trade input
new_trade = pd.DataFrame([[20, 1200, 5]], columns=['FantasyPoints', 'LastSeasonTotalPoints', 'Consistency'])
trade_likelihood = model.predict_proba(new_trade)[:, 1]
print(f"Likelihood of trade being accepted: {trade_likelihood[0] * 100:.2f}%")
